---
# Data Mining - Project 3 - Spring Semester 2020
---
##  Iglezou Myrto  | sdi1700038
---

In [97]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import sys # only needed to determine Python version number
import matplotlib # only needed to determine Matplotlib version number
import nltk
from pandas import DataFrame, read_csv
from nltk import word_tokenize, BigramCollocationFinder
from nltk.corpus import stopwords as stopwords2

In [199]:
trainPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\train.csv" 
testPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\impermium_verification_set.csv" 
y_testPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\impermium_verification_labels.csv"
tempfile = pd.read_csv(trainPath,index_col=False)
traindf = pd.DataFrame(data=tempfile)
tempfile = pd.read_csv(testPath,index_col=False)
testdf = pd.DataFrame(data=tempfile)
tempfile = pd.read_csv(y_testPath,index_col=False)
y_testdf = pd.DataFrame(data=tempfile)
y_test = y_testdf["Insult"]

In [129]:
from string import punctuation
from wordcloud import STOPWORDS,WordCloud

punctuation = list(punctuation)
punctuation.append('’')
stopwordsPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\english_stopwords.txt"
file = open(stopwordsPath,"r",errors='ignore')
stopwords_fromHub = [x.replace('\n',"") for x in file.readlines()]
stopwords = set(STOPWORDS)
stopwords_nltk =  set(stopwords2.words('english'))
stopwords.update(stopwords_nltk)
stopwords.update(set(stopwords_fromHub))

punctuation.append("``")
punctuation.append("...")

In [100]:
def vectorizeData(X_train,X_test,vectorizer):
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    return X_train,X_test

In [101]:
from sklearn import metrics

def classifier(NB,x_train, y_train,x_test,y_test):
    clf = NB
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    train_accuracy = metrics.accuracy_score(y_test, y_pred)
    train_f1 = metrics.f1_score(y_test, y_pred)
    print("Accuracy score = ",train_accuracy)
    print("F1 score = ",train_f1)

# Cleaning Data

In [200]:
import re
from nltk.stem import WordNetLemmatizer

testdf['Comment'] = testdf['Comment'].str.replace('http\S+|www.\S+', '', case=False)
traindf['Comment'] = traindf['Comment'].str.replace('http\S+|www.\S+', '', case=False)

x_train = traindf['Comment']
y_train = traindf['Insult']
x_test = testdf['Comment']

def cleanText(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        text = re.sub("[^a-zA-Z]"," ",text) # matches everything from numbers and punctuations etc , leaving only the words
        for word in word_tokenize(text):
            word = word.lower()
            if (word not in punctuation) and word.isalpha():
                textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

def lemmatize(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        for word in word_tokenize(text):
            word = WordNetLemmatizer().lemmatize(word)
            textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

def removeStopWords(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        for word in word_tokenize(text):
            if word not in stopwords:
                textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

In [201]:
cleanText(x_train)
cleanText(x_test)

---
# Naive Bayes
---

In [34]:
y_test = y_test.tolist()
y_train = y_train.tolist()

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import GaussianNB

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.519910514541387
F1 score =  0.513819664703217


## 1. Lemmatization

In [40]:
lemmatize(x_train)
lemmatize(x_test)

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.5185682326621924
F1 score =  0.5209260908281389


## 2. Stopwords

In [36]:
count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1,stop_words=stopwords)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.5212527964205816
F1 score =  0.5286343612334803


## 3. Bigrams

In [37]:
count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1,ngram_range=(2,2))) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.5592841163310962
F1 score =  0.5248432223830198


## 4. Laplace Smoothing

In [38]:
from sklearn.naive_bayes import MultinomialNB

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer()) 

count_train = count_train.toarray()
count_test = count_test.toarray()
classifier(GaussianNB(var_smoothing=1),count_train, y_train,count_test,y_test)

Accuracy score =  0.48322147651006714
F1 score =  0.6486157590508063


## 5. Try all together

In [39]:
lemmatize(x_train)
lemmatize(x_test)

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1,ngram_range=(2,2),stop_words=stopwords)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(var_smoothing=1),count_train, y_train,count_test,y_test)

Accuracy score =  0.4818791946308725
F1 score =  0.6503623188405797


### Part-of-Speech 

In [202]:
def pos_tag(df,data):
    listOfpos = []  
    for text in data:
        textWords =  word_tokenize(text)
        listOfpos.append(nltk.pos_tag(textWords))

    nounlist = []
    verblist = []
    adverblist = []
    adjectivelist = []

    for list1 in listOfpos:   
        count=0 
        noun=0
        verb=0
        adverb=0
        adjective=0
        for tuple1 in list1:
            count+=1
            verb += tuple1.count('VBD')
            noun += tuple1.count('NN')
            adverb += tuple1.count('VBG')
            adjective  += tuple1.count('JJ') + tuple1.count('JJS')
        if(count>0):
            adverblist.append(adverb/count) # fractionAdverbs
            verblist.append(verb/count) # fractionVerbs
            adjectivelist.append(adjective/count) # fractionAdjectives
            nounlist.append(noun/count) # fractionNouns 
        else:
            adverblist.append(0) # fractionAdverbs
            verblist.append(0) # fractionVerbs
            adjectivelist.append(0) # fractionAdjectives
            nounlist.append(0) # fractionNouns 
    df['fractionNouns'] = nounlist
    df['fractionVerbs'] = verblist
    df['fractionAdverbs'] = adverblist
    df['fractionAdjectives'] = adjectivelist                    
   

In [203]:
train_df = DataFrame(data=x_train)
test_df  = DataFrame(data=x_test)

pos_tag(train_df,x_train)
pos_tag(test_df,x_test)

In [207]:
nouns_train = train_df['fractionNouns']
nouns_test = test_df['fractionNouns']
nouns_tfidf_train,nouns_tfidf_test = vectorizeData(x_train,x_test,TfidfVectorizer(max_df=1.0,min_df=1))
sp_train =  sp.coo_matrix(list(nouns_train))
sp_test =  sp.coo_matrix(list(nouns_test))
X_train = sp.hstack((nouns_tfidf_train, sp_train))
X_test = sp.hstack((nouns_tfidf_test, sp_test))


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 1, expected 3947.

3947


In [157]:
verbs_train = train_df['fractionVerbs']
verbs_test = test_df['fractionVerbs']
verbs_tfidf_train,verbs_tfidf_test = vectorizeData(verbs_train.astype('str'),verbs_test.astype('str'),TfidfVectorizer(max_df=1.0,min_df=1))

In [158]:
adverbs_train = train_df['fractionAdverbs']
adverbs_test = test_df['fractionAdverbs']
adverbs_tfidf_train,adverbs_tfidf_test = vectorizeData(adverbs_train.astype('str'),adverbs_test.astype('str'),TfidfVectorizer(max_df=1.0,min_df=1))

In [159]:
adjectives_train = train_df['fractionAdjectives']
adjectives_test = test_df['fractionAdjectives']
adjectives_tfidf_train,adjectives_tfidf_test = vectorizeData(adjectives_train.astype('str'),adjectives_test.astype('str'),TfidfVectorizer(max_df=1.0,min_df=1))

--- 
# Random Forest Classifier
---

In [175]:
from sklearn.ensemble import RandomForestClassifier
classifier(RandomForestClassifier(),X_train,y_train,X_test,y_test)

Accuracy score =  0.5243847874720358
F1 score =  0.06836108676599474


---
# Support Vector Machine (SVM)
---

In [174]:
from sklearn import svm

classifier(svm.SVC(),X_train,y_train,X_test,y_test)

Accuracy score =  0.5217002237136465
F1 score =  0.04808548530721282
